In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
def gen_data(nobs=1000, a=.5,k=2):
    x = np.random.normal(scale=1., size=(nobs,k))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x.sum(axis=1) + e
    return y,x,nobs


def add_constant(x):
    nobs = x.shape[0]
    x = np.concatenate( (np.ones(shape=(nobs,1)),x) ,axis=1)
    return x


def create_biased_moments(y,x,bias):
    
    #initial setup
    x = add_constant(x)
    nobs, k = x.shape
    xx = (x.T).dot(x)
        
    #setup gradient
    G = np.concatenate((xx,np.zeros((1,k))),axis=0)
    G = np.concatenate((G,np.zeros((k+1,1))),axis=1)
    G[k,k] =1
    
    #calculate sensativity. for ols its just G^{-1}
    L = np.linalg.inv( G.dot(G) ).dot(G) 

    #compute value of moments
    xy = x.T.dot(y) + bias
    beta = np.linalg.inv( xx ).dot( xy )
    y_hat = x.dot(beta)
    mse = ((y - y_hat)**2).mean()
    
    g = np.concatenate( (xy,[mse])  )
    return L.dot(g)
    

#various setup    
y,x,nobs = gen_data()

#first see unbiased
bias = np.array([0, 0,0])
theta_bias = create_biased_moments(y,x,bias)
print('no bias     ', theta_bias)

#bias in x1
bias = np.array([0, 1*nobs,0])
theta_bias = create_biased_moments(y,x,bias)
print('bias in x1  ', theta_bias)

#bias in x2
bias = np.array([0, 0,1*nobs])
theta_bias = create_biased_moments(y,x,bias)
print('bias in x2  ', theta_bias)

#bias in both
bias = np.array([0, 1*nobs,1*nobs])
theta_bias = create_biased_moments(y,x,bias)
print('bias in both', theta_bias)

bias = np.array([0, 1*nobs,-1*nobs])
theta_bias = create_biased_moments(y,x,bias)
print('bias in both', theta_bias)

no bias      [1.01296881 0.46494076 0.47872747 0.95316245]
bias in x1   [1.03811972 1.50909724 0.47042471 1.99731893]
bias in x2   [1.07794647 0.456638   1.4229696  1.89740458]
bias in both [1.10309738 1.50079448 1.41466684 2.92495553]
bias in both [ 0.97314207  1.51740001 -0.47381742  2.95816658]
